## [Impremium Verification labels](edrfsh)
<br> By Cheta Uwadinma

# Problem Statement
This is a model development program, to check if the comment used on this website are insultive or not.

In this project, we will build a model to help predict wheather the comments are insultive or not

In [6]:
#import the Lib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
sns.set_palette(palette='pastel')
import warnings
warnings.filterwarnings("ignore")

In [7]:
#Load the dataset
df = pd.read_csv(r"C:\Users\HomePC\Desktop\impermium_verification_labels.csv")
df.head()

,id,Insult,Date,Comment,Usage
0,1,0,20120603163526Z,"""like this if you are a tribe fan""",PrivateTest
1,2,1,20120531215447Z,"""you're idiot.......................""",PrivateTest
2,3,1,20120823164228Z,"""I am a woman Babs, and the only ""war on women...",PrivateTest
3,4,1,20120826010752Z,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",PrivateTest
4,5,1,20120602223825Z,"""haha green me red you now loser whos winning ...",PrivateTest


In [8]:
#select certain cols
new_df = df[['Insult', 'Comment']]
print (new_df)

      Insult                                            Comment
0          0                 "like this if you are a tribe fan"
1          1              "you're idiot......................."
2          1  "I am a woman Babs, and the only "war on women...
3          1  "WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...
4          1  "haha green me red you now loser whos winning ...
...      ...                                                ...
2230       0  "FUCKIN LAME COME ON WTF STOP FUCKING OVER MY ...
2231       1  "YOU SHUT YOUR IGNORANT PIE HOLE YOU LITTLE IN...
2232       0  "sweetie pie is looking very much like her cou...
2233       1  "ball4real where are you with your miami g-ayn...
2234       0  "Man....if you are a 3 point shooter, you must...

[2235 rows x 2 columns]


In [9]:
#drop some colunms
df = df.drop(columns=['id', 'Date', 'Usage'])

In [10]:
#check data frame
df.head()

,Insult,Comment
0,0,"""like this if you are a tribe fan"""
1,1,"""you're idiot......................."""
2,1,"""I am a woman Babs, and the only ""war on women..."
3,1,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F..."
4,1,"""haha green me red you now loser whos winning ..."


In [11]:
df.describe()

,Insult
count,2235.000000
mean,0.481879
std,0.499783
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2235 entries, 0 to 2234
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Insult   2235 non-null   int64 
 1   Comment  2235 non-null   object
dtypes: int64(1), object(1)
memory usage: 35.1+ KB


In [13]:
df.isnull()

,Insult,Comment
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
2230,False,False
2231,False,False
2232,False,False
2233,False,False


In [14]:
#define the map values
Insult_map = {0: 'Clean', 1 : 'offensive'}
df['Insult'] = df['Insult'].map(Insult_map)

In [15]:
df.head()

,Insult,Comment
0,Clean,"""like this if you are a tribe fan"""
1,offensive,"""you're idiot......................."""
2,offensive,"""I am a woman Babs, and the only ""war on women..."
3,offensive,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F..."
4,offensive,"""haha green me red you now loser whos winning ..."


## Cleaning database


In [16]:
#import the lib for cleaning
import nltk
import re
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer('english')
#from nltk.stem import porterstemmer
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# define a function to clean text
def clean(Comment):
    text = str(Comment).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
df["Comment"].apply(clean)

0                                          like tribe fan
1                                              your idiot
2       woman bab war women see come jackazz like  don...
3              wow  benefit mani win year bat nice stupid
4                     haha green red loser whos win moron
                              ...                        
2230                 fuckin lame come wtf stop fuck biti 
2231      shut ignor pie hole littl insignificunt insect 
2232    sweeti pie look much like cousin maggi ewhat b...
2233                                            miami gay
2234       manif  point shooter must love play wade bron 
Name: Comment, Length: 2235, dtype: object

## Feature Selection

In [18]:
# Transform text data into numerical data
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
import pickle

In [19]:
x = np.array(df['Comment']) 
y = np.array(df['Insult'])

In [20]:
# Transform texts to vectors
cv = CountVectorizer()

In [21]:
# Fit cv on x
x = cv.fit_transform(x)

In [22]:
# Save the CountVectorizer object to a file
with open('cv.pkl', 'wb') as f:
    pickle.dump(cv, f)

In [23]:
# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [24]:
# print the shape
print(f"Shape of x_train: {x_train.shape}")
print(f"Shape of x_test: {x_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

Shape of x_train: (1788, 7645)
Shape of x_test: (447, 7645)
Shape of y_train: (1788,)
Shape of y_test: (447,)


## Build the model

In [25]:
# import the algorithm library
# model selection
#from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
#from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,f1_score
# using naive bayes 
model = MultinomialNB()
# fit the model to the set
model.fit(x_train, y_train)

MultinomialNB()

In [26]:
# make y prediction
y_pred = model.predict(x_test)

In [27]:
# print classification report
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

       Clean       0.46      0.77      0.57       134
   offensive       0.86      0.61      0.71       313

    accuracy                           0.66       447
   macro avg       0.66      0.69      0.64       447
weighted avg       0.74      0.66      0.67       447



In [28]:
# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [29]:
# import random forest classifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,f1_score

In [30]:
svc = SVC(kernel='linear', random_state=42)

In [31]:
model = svc.fit(x_train, y_train)



In [32]:
model

SVC(kernel='linear', random_state=42)

In [33]:
# make prediction
svc_pred = model.predict(x_test)
print(classification_report(svc_pred, y_test))

              precision    recall  f1-score   support

       Clean       0.74      0.71      0.72       235
   offensive       0.69      0.72      0.71       212

    accuracy                           0.71       447
   macro avg       0.71      0.71      0.71       447
weighted avg       0.71      0.71      0.71       447



In [63]:
# test the model with user input
user =  input('enter a sentence: ')
data = cv.transform([user]).toarray()
output = model.predict(data)
print(user, '', output)

you fool  ['offensive']


In [62]:
#save the Model
with open ('model.pkl', 'wb') as f:
    pickle.dump(model, f)